In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, time, timedelta
from tqdm import tqdm

In [3]:
teams = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BRK",
    "Charlotte Hornets": "CHO",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

### Get HTML for Boxscore

In [4]:
# Send a GET request to the webpage
url_team = 'https://www.basketball-reference.com/teams/MIA/2022_games.html'
response_team = requests.get(url_team)

# Parse the HTML content using BeautifulSoup
soup_team = BeautifulSoup(response_team.content, 'html.parser')

# Find all the links that contain 'boxscores' in their href attribute
boxscore_links = []
for link in soup_team.find_all('a', href=True):
    if 'boxscores' in link['href'] and 'html' in link['href'] :
        a=link['href'].split('/',2)[2]
        boxscore_links.append('https://www.basketball-reference.com/boxscores/pbp/'+a)

# Print the list of box score links
boxscore_links

['https://www.basketball-reference.com/boxscores/pbp/202110210MIA.html',
 'https://www.basketball-reference.com/boxscores/pbp/202110230IND.html',
 'https://www.basketball-reference.com/boxscores/pbp/202110250MIA.html',
 'https://www.basketball-reference.com/boxscores/pbp/202110270BRK.html',
 'https://www.basketball-reference.com/boxscores/pbp/202110290MIA.html',
 'https://www.basketball-reference.com/boxscores/pbp/202110300MEM.html',
 'https://www.basketball-reference.com/boxscores/pbp/202111020DAL.html',
 'https://www.basketball-reference.com/boxscores/pbp/202111040MIA.html',
 'https://www.basketball-reference.com/boxscores/pbp/202111060MIA.html',
 'https://www.basketball-reference.com/boxscores/pbp/202111080DEN.html',
 'https://www.basketball-reference.com/boxscores/pbp/202111100LAL.html',
 'https://www.basketball-reference.com/boxscores/pbp/202111110LAC.html',
 'https://www.basketball-reference.com/boxscores/pbp/202111130UTA.html',
 'https://www.basketball-reference.com/boxscores/pb

### Get HTML of All Box Score

In [5]:
def data_extraction(x):
    df = pd.DataFrame()
    r = requests.get(x)
    soup = BeautifulSoup(r.content, 'html.parser')
    # Extract the table with play-by-play data
    pbp_table = soup.find_all('table')[0]
    strong=soup.find_all('strong')
    team_A=strong[1].get_text()
    team_A=team_A.replace('\n', '')
    team_B=strong[2].get_text()
    team_B=team_B.replace('\n', '')

    team_A=teams[team_A]
    team_B=teams[team_B]

    date=(strong[0].get_text().split(',',1)[1]).strip()

    for row in pbp_table.find_all("tr"):
        tds = row.find_all("td")
        if len(tds) > 0:
            time = tds[0].get_text()
            if row.find("td", {"colspan": "5"}) or row.find("td", {"colspan": "6"}):
                event = tds[1].get_text()
                df = df.append({"Team": '', 
                                "Time": time,
                                "Raw": event,
                                "Team_A_Score":'',
                                "Team_B_Score":''}, ignore_index=True)
            else:
                score = tds[3].get_text()
                score_A,score_B=score.split('-')
                if tds[1].get_text().strip()=="":
                    event =tds[5].get_text()
                    team=team_B
                else:
                    event = tds[1].get_text()
                    team=team_A
                df = df.append({"Team": team, 
                                "Time": time,
                                "Raw": event,
                                "Team_A_Score":score_A,
                                "Team_B_Score":score_B}, ignore_index=True)
    df['Date']= date
    df['Team_A']=team_A
    df['Team_B']=team_B
    df['Time'] = pd.to_datetime(df['Time'], format='%M:%S.%f')
    conditions=[(df["Raw"].str.contains("End of 1st quarter")),(df["Raw"].str.contains("End of 2nd quarter")),
                (df["Raw"].str.contains("End of 3rd quarter")),(df["Raw"].str.contains("End of 4th quarter")),
               (df["Raw"].str.contains("End of 1st overtime")),(df["Raw"].str.contains("End of 2nd overtime")),
               (df["Raw"].str.contains("End of 3rd overtime")),(df["Raw"].str.contains("End of 4th overtime")),]
    outcomes=['Q1','Q2','Q3','Q4','OT1','OT2','OT3','OT4']
    default=pd.NaT
    df['Q']=np.select(conditions, outcomes,default=default)
    df['Q']=df['Q'].fillna(method='backfill')
    ot4 = pd.to_datetime('1:07:00.0', format='%H:%M:%S.%f')
    ot3 = pd.to_datetime('1:02:00.0', format='%H:%M:%S.%f')
    ot2 = pd.to_datetime('00:57:00.0', format='%H:%M:%S.%f')
    ot1 = pd.to_datetime('00:52:00.0', format='%H:%M:%S.%f')
    dt48 = pd.to_datetime('00:48:00.0', format='%H:%M:%S.%f')
    dt36 = pd.to_datetime('00:36:00.0', format='%H:%M:%S.%f')
    dt24 = pd.to_datetime('00:24:00.0', format='%H:%M:%S.%f')
    dt12 = pd.to_datetime('00:12:00.0', format='%H:%M:%S.%f')


    mask1 = df['Q'] == 'Q1'
    df.loc[mask1, 'Time_48'] = dt12 - df.loc[mask1, 'Time']
    mask2 = df['Q'] == 'Q2'
    df.loc[mask2, 'Time_48'] = dt24 - df.loc[mask2, 'Time']
    mask3 = df['Q'] == 'Q3'
    df.loc[mask3, 'Time_48'] = dt36 - df.loc[mask3, 'Time']
    mask4 = df['Q'] == 'Q4'
    df.loc[mask4, 'Time_48'] = dt48 - df.loc[mask4, 'Time']
    mask5 = df['Q'] == 'OT1'
    df.loc[mask5, 'Time_48'] = ot1 - df.loc[mask5, 'Time']
    mask6 = df['Q'] == 'OT2'
    df.loc[mask6, 'Time_48'] = ot2 - df.loc[mask6, 'Time']
    mask7 = df['Q'] == 'OT3'
    df.loc[mask7, 'Time_48'] = ot3 - df.loc[mask7, 'Time']
    mask8 = df['Q'] == 'OT4'
    df.loc[mask8, 'Time_48'] = ot4 - df.loc[mask8, 'Time']

    df["Team_A_Tech"] = np.where((df["Raw"].str.contains("Technical foul"))& (df["Team"] == team_A), 1, 0)
    df["Team_B_Tech"] = np.where((df["Raw"].str.contains("Technical foul"))& (df["Team"] == team_B), 1, 0)
    df["Team_A_PF"] = np.where((df["Raw"].str.contains("foul")) & (df["Team"] == team_B) & (df['Team_B_Tech'] != 1), 1, 0)
    df["Team_B_PF"] = np.where((df["Raw"].str.contains("foul")) & (df["Team"] == team_A) & (df['Team_A_Tech'] != 1), 1, 0)
    df["Team_A_FTM"] = np.where((df["Raw"].str.contains("makes free throw"))& (df["Team"] == team_A), 1, 0)
    df["Team_A_FTA"]= np.where((df["Raw"].str.contains("free throw"))& (df["Team"] == team_A), 1, 0)
    df["Team_B_FTM"] = np.where((df["Raw"].str.contains("makes free throw"))& (df["Team"] == team_B), 1, 0)
    df["Team_B_FTA"]= np.where((df["Raw"].str.contains("free throw"))& (df["Team"] == team_B), 1, 0)
    df["Team_A_FG"]= np.where(((df["Raw"].str.contains("makes 2-pt"))|(df["Raw"].str.contains("makes 3-pt")))& (df["Team"] == team_A), 1, 0)
    df["Team_A_FGA"]= np.where(((df["Raw"].str.contains("2-pt"))|(df["Raw"].str.contains("3-pt")))& (df["Team"] == team_A), 1, 0)
    df["Team_B_FG"]= np.where(((df["Raw"].str.contains("makes 2-pt"))|(df["Raw"].str.contains("makes 3-pt")))& (df["Team"] == team_B), 1, 0)
    df["Team_B_FGA"]= np.where(((df["Raw"].str.contains("2-pt"))|(df["Raw"].str.contains("3-pt")))& (df["Team"] == team_B), 1, 0)
    df["Team_A_3P"]= np.where((df["Raw"].str.contains("makes 3-pt"))& (df["Team"] == team_A), 1, 0)
    df["Team_A_3PA"]= np.where((df["Raw"].str.contains("3-pt"))& (df["Team"] == team_A), 1, 0)
    df["Team_B_3P"]= np.where((df["Raw"].str.contains("makes 3-pt"))& (df["Team"] == team_B), 1, 0)
    df["Team_B_3PA"]= np.where((df["Raw"].str.contains("3-pt"))& (df["Team"] == team_B), 1, 0)
    df["Team_A_BLK"] = np.where((df["Raw"].str.contains("block")) & (df["Team"] == team_B), 1, 0)
    df["Team_B_BLK"] = np.where((df["Raw"].str.contains("block")) & (df["Team"] == team_A), 1, 0)
    df["Team_A_ST"] = np.where((df["Raw"].str.contains("steal")) & (df["Team"] == team_B), 1, 0)
    df["Team_B_ST"] = np.where((df["Raw"].str.contains("steal")) & (df["Team"] == team_A), 1, 0)
    df["Team_A_TO"] = np.where((df["Raw"].str.contains("Turnover")) & (df["Team"] == team_A), 1, 0)
    df["Team_B_TO"] = np.where((df["Raw"].str.contains("Turnover")) & (df["Team"] == team_B), 1, 0)
    df["Team_A_AST"] = np.where((df["Raw"].str.contains("assist")) & (df["Team"] == team_A), 1, 0)
    df["Team_B_AST"] = np.where((df["Raw"].str.contains("assist")) & (df["Team"] == team_B), 1, 0)
    df["Team_A_teamORB"] = np.where((df["Raw"].str.contains("Offensive rebound by Team")) & (df["Team"] == team_A), 1, 0)
    df["Team_B_teamORB"] = np.where((df["Raw"].str.contains("Offensive rebound by Team")) & (df["Team"] == team_B), 1, 0)
    df["Team_A_teamDRB"] = np.where((df["Raw"].str.contains("Defensive rebound by Team")) & (df["Team"] == team_A), 1, 0)
    df["Team_B_teamDRB"] = np.where((df["Raw"].str.contains("Defensive rebound by Team")) & (df["Team"] == team_B), 1, 0)
    df["Team_A_ORB"] = np.where((df["Raw"].str.contains("Offensive rebound")) & (df["Team"] == team_A) & (df['Team_A_teamORB'] != 1), 1, 0)
    df["Team_B_ORB"] = np.where((df["Raw"].str.contains("Offensive rebound")) & (df["Team"] == team_B) & (df['Team_B_teamORB'] != 1), 1, 0)
    df["Team_A_DRB"] = np.where((df["Raw"].str.contains("Defensive rebound")) & (df["Team"] == team_A) & (df['Team_A_teamDRB'] != 1), 1, 0)
    df["Team_B_DRB"] = np.where((df["Raw"].str.contains("Defensive rebound")) & (df["Team"] == team_B) & (df['Team_B_teamDRB'] != 1), 1, 0)

    df_cumsum = df.iloc[:,10:].cumsum()
    df_final = pd.concat([df[['Date','Time_48','Q','Team_A','Team_B','Team_A_Score','Team_B_Score']],df_cumsum], axis=1)
    df_final['Team_A_Score'] = df_final['Team_A_Score'].replace(r'^\s*$', np.nan, regex=True).ffill()
    df_final['Team_B_Score'] = df_final['Team_B_Score'].replace(r'^\s*$', np.nan, regex=True).ffill()
    df_final['Team_A_Score'] = df_final['Team_A_Score'].replace(r'^\s*$', np.nan, regex=True).bfill()
    df_final['Team_B_Score'] = df_final['Team_B_Score'].replace(r'^\s*$', np.nan, regex=True).bfill()
    df_final['Result'] = np.where((df_final['Team_A_Score'].iloc[-1]>df_final['Team_B_Score'].iloc[-1]),'Team A','Team B')
    return df_final

In [6]:
empty=pd.DataFrame()
game=1
for x in tqdm(boxscore_links):
    data=data_extraction(x)
    data['Game']=game
    game+=1
    empty=pd.concat([empty,data])

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:34<00:00,  3.34s/it]


In [7]:
empty

,Date,Time_48,Q,Team_A,Team_B,Team_A_Score,Team_B_Score,Team_A_Tech,Team_B_Tech,Team_A_PF,...,Team_A_teamORB,Team_B_teamORB,Team_A_teamDRB,Team_B_teamDRB,Team_A_ORB,Team_B_ORB,Team_A_DRB,Team_B_DRB,Result,Game
0,"October 21, 2021",00:00:00,Q1,MIL,MIA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Team A,1
1,"October 21, 2021",00:00:14,Q1,MIL,MIA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Team A,1
2,"October 21, 2021",00:00:20,Q1,MIL,MIA,0,2,0,0,0,...,0,0,0,0,0,0,0,0,Team A,1
3,"October 21, 2021",00:00:38,Q1,MIL,MIA,0,2,0,0,0,...,0,0,0,0,0,0,0,0,Team A,1
4,"October 21, 2021",00:00:38,Q1,MIL,MIA,0,2,0,0,0,...,0,0,0,0,0,0,0,0,Team A,1
5,"October 21, 2021",00:00:38,Q1,MIL,MIA,0,2,0,0,0,...,1,0,0,0,0,0,0,0,Team A,1
6,"October 21, 2021",00:00:38,Q1,MIL,MIA,1,2,0,0,0,...,1,0,0,0,0,0,0,0,Team A,1
7,"October 21, 2021",00:00:50,Q1,MIL,MIA,1,2,0,0,1,...,1,0,0,0,0,0,0,0,Team A,1
8,"October 21, 2021",00:00:50,Q1,MIL,MIA,1,3,0,0,1,...,1,0,0,0,0,0,0,0,Team A,1
9,"October 21, 2021",00:00:50,Q1,MIL,MIA,1,3,0,0,1,...,1,0,0,0,0,0,0,0,Team A,1


In [136]:
empty.to_csv('MIA-2022.csv')